In [13]:
import pandas as pd
product = pd.read_excel("/content/Product Datasets.xlsx")
product

,productid,product_name,product_category_tree,brand,retail_price,discounted_price
0,1,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Alisha,999.0,379.0
1,2,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor,32157.0,22646.0
2,3,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",AW,999.0,499.0
3,4,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Alisha,699.0,267.0
4,5,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Sicons,220.0,210.0
...,...,...,...,...,...,...
19996,103673,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Wallmantra,1429.0,1143.0
19997,103676,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Elite Collection,1299.0,999.0
19998,103681,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Elite Collection,1499.0,1199.0
19999,103683,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Elite Collection,1499.0,999.0


In [14]:
import re

def clean_product_name(product_name):
  return re.sub("[^a-zA-Z0-9 ]","",str(product_name))

product["clean_product_name"] = product["product_name"].apply(clean_product_name)
product = product.dropna()
product

,productid,product_name,product_category_tree,brand,retail_price,discounted_price,clean_product_name
0,1,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Alisha,999.0,379.0,Alisha Solid Womens Cycling Shorts
1,2,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed
2,3,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",AW,999.0,499.0,AW Bellies
3,4,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Alisha,699.0,267.0,Alisha Solid Womens Cycling Shorts
4,5,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Sicons,220.0,210.0,Sicons All Purpose Arnica Dog Shampoo
...,...,...,...,...,...,...,...
19995,103671,WallDesign Small Vinyl Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",WallDesign,1500.0,730.0,WallDesign Small Vinyl Sticker
19996,103673,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Wallmantra,1429.0,1143.0,Wallmantra Large Vinyl Stickers Sticker
19997,103676,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Elite Collection,1299.0,999.0,Elite Collection Medium Acrylic Sticker
19998,103681,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Elite Collection,1499.0,1199.0,Elite Collection Medium Acrylic Sticker


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(product["clean_product_name"])

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_product_name(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = product.iloc[indices].iloc[::-1]
    
    return results

In [17]:
import ipywidgets as widgets
from IPython.display import display

product_input = widgets.Text(
    value='Type Here',
    description='What product you Want me Recommed:',
    disabled=False
)
product_list = widgets.Output()

def on_type(data):
    with product_list:
        product_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

product_input.observe(on_type, names='value')


display(product_input, product_list)

Text(value='Type Here', description='What product you Want me Recommed:')

Output()

In [18]:
product_id = 89745

products = product[product["productid"] == product_id]

In [20]:
ratings = pd.read_excel("/content/Product Review.xlsx")
ratings

,userId,productid,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
1048570,7045,4447,3.5
1048571,7045,4720,4.0
1048572,7045,4857,4.0
1048573,7045,4886,5.0


In [21]:
ratings.dtypes

userId         int64
productid      int64
rating       float64
dtype: object

In [22]:
similar_users = ratings[(ratings["productid"] == product_id) & (ratings["rating"] > 4)]["userId"].unique()

In [23]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["productid"]


In [24]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [25]:
all_users = ratings[(ratings["productid"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [26]:
all_user_recs = all_users["productid"].value_counts() / len(all_users["userId"].unique())

In [27]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [28]:
rec_percentages

,similar,all
1,0.231373,0.124122
47,0.203922,0.145199
50,0.196078,0.206714
110,0.145098,0.158938
260,0.372549,0.214676
...,...,...
135143,0.101961,0.009368
152081,0.141176,0.020921
164179,0.121569,0.025449
166528,0.133333,0.013271


In [29]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [30]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [32]:
rec_percentages.head(10).merge(product, left_index=True, right_on="productid")

,similar,all,score,productid,product_name,product_category_tree,brand,retail_price,discounted_price,clean_product_name
17067,1.000000,0.039813,25.117647,89745,Chrishan Alloy Bracelet,"[""Jewellery >> Bangles, Bracelets & Armlets >>...",Chrishan,699.0,449.0,Chrishan Alloy Bracelet
16725,0.176471,0.009992,17.660846,88140,Jogur Men's Solid Formal Shirt,"[""Clothing >> Men's Clothing >> Shirts >> Form...",Slim,899.0,629.0,Jogur Mens Solid Formal Shirt


In [37]:
def find_similar_product(product_id):
    similar_users = ratings[(ratings["productid"] == product_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["productid"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["productid"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["productid"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(product, left_index=True, right_on="productid")[["score", "product_name", "brand", "retail_price", "discounted_price"]]

In [38]:
import ipywidgets as widgets
from IPython.display import display

product_name_input = widgets.Text(
    value='Type Here',
    description='Product Name:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            product_id = results.iloc[0]["productid"]
            display(find_similar_product(product_id))

product_name_input.observe(on_type, names='value')

display(product_name_input, recommendation_list)

Text(value='Type Here', description='Product Name:')

Output()